# Linear regression ecc_size

In [12]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# from plot_utils import plotly_template
import plotly.express as px

# import data base
main_dir = '/home/mchevillard/disks/meso_shared/gaze_exp/'
pp_dir = "{}/derivatives/pp_data".format(main_dir)
tsv_dir = '{}/sub-all/prf/tsv'.format(pp_dir)

data = pd.read_table('{}/group_task-prf_prf.tsv'.format(tsv_dir))
data = data.drop(columns=['Unnamed: 0'])
data_loo = pd.read_table('{}/group_task-loo_prf_prf.tsv'.format(tsv_dir))
data_loo = data_loo.drop(columns=['Unnamed: 0'])

In [13]:
# define parameters
ecc_th = [0,15]
size_th= [0.1,20]
r2_th = 0.1

# Replace all data outer threshold with NaN data
data.loc[(data.ecc < ecc_th[0]) | (data.ecc > ecc_th[1]) | (data.sd < size_th[0]) | (data.sd > size_th[1]) | (data.r2 <=r2_th)] = np.nan
data = data.dropna()
data_loo.loc[(data_loo.ecc<ecc_th[0]) | (data_loo.ecc > ecc_th[1]) | (data_loo.sd < size_th[0]) | (data_loo.sd > size_th[1]) | (data_loo.r2 <=r2_th)] = np.nan
data_loo = data_loo.dropna()

In [14]:
# np.mean(data.loc[(data.subject =='sub-001')&(data.roi =='V1'),'r2'])

In [15]:
rois = pd.unique(data.roi)
mask = pd.notnull(data.subject)
subjects = pd.unique(data.subject[mask])
rois_loo = pd.unique(data_loo.roi)
mask_loo = pd.notnull(data_loo.subject)
subjects_loo = pd.unique(data_loo.subject[mask_loo])

In [48]:
fig_height, fig_width = 400, 1200

In [23]:
# roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]
roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

In [24]:
#figures
fig_subplot_ecc_1 = go.Figure()
fig_subplot_ecc_2 = go.Figure()

# Old single plots

In [21]:
#subplot settings SIZE
rows, cols, = 1,2
fig_height, fig_width = 1080,1920

fig = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1)

roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        if i ==1 :
            showlegend = True
        else :
            showlegend = False
        col = (i % 2) + 1
        row = i // 2 +1
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        slope, intercept = np.polyfit(df.ecc,df.sd,deg=1, w=df.r2)
        line = slope * df.ecc + intercept
        
        fig.add_trace(go.Scatter(x=df.ecc, y=line, mode='lines',name=roi, legendgroup=roi,line_color = roi_colors[j],showlegend=showlegend),row=row, col=col)
        
        
        # Set axis titles only for the left-most column and bottom-most row
        if col == 1:
            fig.update_yaxes(title_text='pRF size (dva)', row=row, col=col)
        if row == 1:
            fig.update_xaxes(title_text='pRF eccentricity (dva)', row=row, col=col)
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0.25, y=16, showarrow=False, text=('{}'.format(subject)), row=1, col=i + 1, font=dict(size=20))
    
fig.update_layout(height=fig_height, width=fig_width,showlegend=True, template='simple_white')

fig.write_image('/home/mchevillard/disks/meso_S/size_ecc.pdf')

In [43]:
#subplot settings R2
rows, cols, = 1,2
fig_height, fig_width = 1080,1920

fig = make_subplots(rows=rows, cols=cols, print_grid=False, vertical_spacing=0.08, horizontal_spacing=0.1)

roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

for i, subject in enumerate(subjects):
    for j, roi in enumerate(rois):
        if i ==1 :
            showlegend = True
        else :
            showlegend = False
        col = (i % 2) + 1
        row = i // 2 +1
        df = data.loc[(data.subject == subject) & (data.roi == roi)]
        slope, intercept = np.polyfit(df.ecc,df.r2, deg=1)
        line = slope * df.ecc + intercept
        
        fig.add_trace(go.Scatter(x=df.ecc, y=line, mode='lines',name=roi, legendgroup=roi,line_color = roi_colors[j],showlegend=showlegend),row=row, col=col)
        
        
        # Set axis titles only for the left-most column and bottom-most row
        if col == col:
            fig.update_yaxes(title_text='R2', range=[0,0.4], row=row, col=col)
        if row == 1:
            fig.update_xaxes(title_text='pRF eccentricity (dva)', row=row, col=col)
        #Add the name of the plot with the subject name
        fig.add_annotation(x=0.25, y=16, showarrow=False, text=('{}'.format(subject)), row=1, col=i + 1, font=dict(size=20))
    
fig.update_layout(height=fig_height, width=fig_width,showlegend=True, template='simple_white')

fig.write_image('/home/mchevillard/disks/meso_S/R2_ecc.pdf')

# Subplot Size-Ecc 3 areas / plot

In [50]:
rows, cols = 1,4

fig_subplot_ecc_1 = make_subplots(rows=rows, cols=cols, print_grid=False)
fig_subplot_ecc_2 = make_subplots(rows=rows, cols=cols, print_grid=False)

roi_colors_loo = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

lines = [['V1', 'V2', 'V3'],['V3AB', 'LO', 'VO'],['hMT+', 'iIPS', 'sIPS'],['iPCS', 'sPCS', 'mPCS']]


for i, subject in enumerate(subjects):
    for l, line_label in enumerate(lines):
        for j, roi in enumerate(line_label):
            fig=fig_subplot_ecc_1 if i==0 else fig_subplot_ecc_2
            #Sorting best datas
            df = data_loo.loc[(data_loo.subject == subject) & (data_loo.roi == roi)]
            df = df.sort_values('r2', ascending=False)
            df = df.head(250)
            #Parametring colors
            roi_color = roi_colors_loo[j + l * 3]
            roi_color_opac = f"rgba{roi_color[3:-1]}, 0.15)"
            
            # Grouping by eccentricities
            df_grouped = df.groupby(pd.cut(df['ecc'], bins=np.arange(0, 17, 2.5)))
            
            df_sorted = df.sort_values('ecc')
            
            ecc_mean = df_grouped['ecc'].mean().tolist()
            sd_mean = df_grouped['sd'].mean().tolist()
            r2_mean = df_grouped['r2'].mean().tolist()
            
            # CI95 for each group of ecc
            ci = df_grouped['sd'].apply(lambda x: stats.t.interval(0.95, len(x) - 1, loc=x.mean(), scale=stats.sem(x)))
            upper_bound = ci.apply(lambda x: x[1])
            lower_bound = ci.apply(lambda x: x[0])
            
            # Linear regression
            slope, intercept = np.polyfit(ecc_mean, sd_mean, deg=1, w=r2_mean)
            slope_upper, intercept_upper = np.polyfit(ecc_mean, upper_bound, deg=1, w=r2_mean)
            slope_lower, intercept_lower = np.polyfit(ecc_mean, lower_bound, deg=1, w=r2_mean)
            line = slope * df_sorted.ecc + intercept
            line_upper = slope_upper * df_sorted.ecc + intercept_upper
            line_lower = slope_lower * df_sorted.ecc + intercept_lower

            fig.add_trace(go.Scatter(x=df_sorted.ecc, y=line, mode='lines', name=roi, legendgroup=roi, line=dict(color=roi_color, width=3), showlegend=False), row=1, col=l + 1)

            # Error area
            fig.add_trace(go.Scatter(x=np.concatenate([df_sorted.ecc, df_sorted.ecc[::-1]]), y=np.concatenate([list(line_upper), list(line_lower[::-1])]), mode='lines', fill='toself', fillcolor=roi_color_opac, line=dict(color=roi_color_opac, width=1), showlegend=False), row=1, col=l + 1)

            #Error bars
            fig.add_trace(go.Scatter(x=ecc_mean, y=sd_mean, mode='markers', error_y=dict(type='data', array=ci.apply(lambda x: (x[1] - x[0]) / 2).tolist(), visible=True, thickness=1, width=0), marker=dict(color=roi_color, size=12), showlegend=False), row=1, col=l + 1)
            
            #Add legend
            annotation = go.layout.Annotation(x=1.2, y=20-j*2, text=roi, showarrow=False, font=dict(color=roi_color, size=12))
            fig.add_annotation(annotation, row=1, col=l+1)
        
        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='pRF size (dva)', row=1, col=1)
        fig.update_xaxes(title_text='pRF eccentricity (dva)', range=[0,15], row=1, col=l+1)
        fig.update_yaxes(range=[0,20])
        fig.update_layout(height=fig_height, width=fig_width, showlegend=False, template='simple_white')


    fig.write_image('/home/mchevillard/disks/meso_S/figures/{}_subplot_size_ecc_250.pdf'.format(subject))

In [ ]:
# fig.add_trace(go.Scatter(x=ecc_mean, y=sd_mean, mode='markers', marker=dict(color=roi_color, size=11), showlegend=False), row=1, col=l + 1)
   
 # fig.add_trace(go.Scatter(x=ecc_mean, y=sd_mean, mode='markers', marker=dict(color=roi_color), showlegend=False), row=1, col=l + 1)
              
  # fig.add_trace(go.Scatter(x=df.ecc, y=(line_upper.tolist() + line_lower.tolist()[::-1]), mode='lines', fill='tonexty', fillcolor=roi_color_opac, line=dict(color=roi_color_opac, width=1), showlegend=False), row=1, col=l + 1)
              
# fig.add_trace(go.Scatter(x=df.ecc, y=line, mode='lines', fill='none', name=roi, legendgroup=roi, line=dict(color=roi_color, width=0), showlegend=False), row=1, col=l + 1)

# # Tracer l'aire entre les limites supérieure et inférieure
            # fig.add_trace(go.Scatter(x=df.ecc, y=upper_bound, mode='lines', fill='tonexty', fillcolor=roi_color_opac, line=dict(color=roi_color, width=0), showlegend=False), row=1, col=l + 1)
            # fig.add_trace(go.Scatter(x=df.ecc, y=lower_bound, mode='lines', fill='tonexty', fillcolor=roi_color_opac, line=dict(color=roi_color, width=0), showlegend=False), row=1, col=l + 1)
           

In [48]:
## faire CI95
# moy +/- std

## feat linéaire std mean haute et mean basse

# Uriel's version

In [1]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# from plot_utils import plotly_template
import plotly.express as px

# import data base
main_dir = '/home/mchevillard/disks/meso_shared/gaze_exp/'
pp_dir = "{}/derivatives/pp_data".format(main_dir)
tsv_dir = '{}/sub-all/prf/tsv'.format(pp_dir)

data = pd.read_table('{}/group_task-prf_prf.tsv'.format(tsv_dir))
data = data.drop(columns=['Unnamed: 0'])

In [15]:
ecc_th = [0,20]
size_th= [0.1,20]
r2_th = 0.1


data.loc[(data.ecc < ecc_th[0]) | (data.ecc > ecc_th[1]) | (data.sd < size_th[0]) | (data.sd > size_th[1]) | (data.r2 <=r2_th )] = np.nan
data = data.dropna()

In [16]:
# subject and roi settings

subject = 'sub-002'
roi = 'V2'

sub_roi = data.loc[(data.subject == subject)&(data.roi == roi)]

In [17]:
slope, intercept = np.polyfit(sub_roi.ecc,sub_roi.sd,deg=1, w=sub_roi.r2)
line = slope * sub_roi.ecc + intercept

In [1]:
# #plot 
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=sub_roi.ecc, y=sub_roi.sd, mode='markers', name=roi))
# fig.add_trace(go.Scatter(x=sub_roi.ecc, y=line, mode='lines', name='linear regression'))
# fig.update_layout(title= '{subject}_{roi}'.format(subject=subject,roi=roi) ,xaxis_title='ecc', yaxis_title='size',xaxis=dict(range=[0, 20]), yaxis=dict(range=[0, 10]))
# fig.show()